In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


In [3]:
train_df = pd.read_csv("/content/fraudTrain.csv")
test_df = pd.read_csv("/content/fraudTest.csv")

df = pd.concat([train_df, test_df], axis=0)

In [4]:
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495.0,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1.325376e+09,36.011293,-82.048315,0.0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149.0,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1.325376e+09,49.159047,-118.186462,0.0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154.0,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1.325376e+09,43.150704,-112.154481,0.0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939.0,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1.325376e+09,47.034331,-112.561071,0.0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99.0,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1.325376e+09,38.674999,-78.632459,0.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 488808 entries, 0 to 252125
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             488808 non-null  int64  
 1   trans_date_trans_time  488808 non-null  object 
 2   cc_num                 488808 non-null  int64  
 3   merchant               488807 non-null  object 
 4   category               488807 non-null  object 
 5   amt                    488807 non-null  float64
 6   first                  488807 non-null  object 
 7   last                   488807 non-null  object 
 8   gender                 488807 non-null  object 
 9   street                 488807 non-null  object 
 10  city                   488807 non-null  object 
 11  state                  488807 non-null  object 
 12  zip                    488807 non-null  float64
 13  lat                    488807 non-null  float64
 14  long                   488807 non-null  f

In [6]:
df.describe()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,488808.000000,4.888080e+05,488807.000000,488807.000000,488807.000000,488807.000000,4.888070e+05,4.888060e+05,488806.000000,488806.000000,488806.000000
mean,122323.489174,4.173906e+17,70.194375,48795.079678,38.539353,-90.208685,8.919783e+04,1.353972e+09,38.538820,-90.208175,0.005994
std,70764.300838,1.309128e+18,154.857996,26871.037302,5.071850,13.737411,3.019884e+05,2.251515e+07,5.107275,13.749964,0.077190
min,0.000000,3.040768e+09,1.000000,1257.000000,20.027100,-165.672300,2.300000e+01,1.325376e+09,19.029798,-166.671575,0.000000
25%,61100.750000,1.800400e+14,9.650000,26237.000000,34.668900,-96.790900,7.430000e+02,1.331323e+09,34.748487,-96.874618,0.000000
50%,122201.500000,3.520550e+15,47.470000,48174.000000,39.371600,-87.461600,2.443000e+03,1.371991e+09,39.366157,-87.421019,0.000000
75%,183302.250000,4.635331e+15,83.205000,72011.000000,41.940400,-80.158000,2.047800e+04,1.375683e+09,41.957998,-80.237229,0.000000
max,252125.000000,4.992346e+18,17897.240000,99783.000000,66.693300,-67.950300,2.906700e+06,1.379773e+09,67.510267,-66.952352,1.000000


In [7]:
df.duplicated()

,0
0,False
1,False
2,False
3,False
4,False
...,...
252121,False
252122,False
252123,False
252124,False


In [8]:
df.isnull().sum()

,0
Unnamed: 0,0
trans_date_trans_time,0
cc_num,0
merchant,1
category,1
amt,1
first,1
last,1
gender,1
street,1


In [9]:
df.drop(columns=[
    'Unnamed: 0', 'trans_date_trans_time',
    'cc_num', 'merchant', 'first', 'last',
    'street', 'city', 'state', 'zip',
    'dob', 'trans_num'
], inplace=True)


In [10]:
df = pd.get_dummies(df, drop_first=True)


In [12]:
df.dropna(subset=['is_fraud'], inplace=True)
X = df.drop("is_fraud", axis=1)
y = df["is_fraud"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [13]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [14]:
lr = LogisticRegression(max_iter=1000, class_weight='balanced')
lr.fit(X_train, y_train)

lr_pred = lr.predict(X_test)

print("Logistic Regression")
print(confusion_matrix(y_test, lr_pred))
print(classification_report(y_test, lr_pred))
print("ROC-AUC:", roc_auc_score(y_test, lr.predict_proba(X_test)[:,1]))


Logistic Regression
[[85466 11710]
 [   83   503]]
              precision    recall  f1-score   support

         0.0       1.00      0.88      0.94     97176
         1.0       0.04      0.86      0.08       586

    accuracy                           0.88     97762
   macro avg       0.52      0.87      0.51     97762
weighted avg       0.99      0.88      0.93     97762

ROC-AUC: 0.9569049058026659


In [15]:
dt = DecisionTreeClassifier(
    max_depth=10,
    class_weight='balanced',
    random_state=42
)
dt.fit(X_train, y_train)

dt_pred = dt.predict(X_test)

print("Decision Tree")
print(confusion_matrix(y_test, dt_pred))
print(classification_report(y_test, dt_pred))
print("ROC-AUC:", roc_auc_score(y_test, dt.predict_proba(X_test)[:,1]))


Decision Tree
[[91757  5419]
 [   30   556]]
              precision    recall  f1-score   support

         0.0       1.00      0.94      0.97     97176
         1.0       0.09      0.95      0.17       586

    accuracy                           0.94     97762
   macro avg       0.55      0.95      0.57     97762
weighted avg       0.99      0.94      0.97     97762

ROC-AUC: 0.9733417178949226


In [16]:
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

rf_pred = rf.predict(X_test)

print("Random Forest")
print(confusion_matrix(y_test, rf_pred))
print(classification_report(y_test, rf_pred))
print("ROC-AUC:", roc_auc_score(y_test, rf.predict_proba(X_test)[:,1]))


Random Forest
[[95387  1789]
 [  131   455]]
              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99     97176
         1.0       0.20      0.78      0.32       586

    accuracy                           0.98     97762
   macro avg       0.60      0.88      0.66     97762
weighted avg       0.99      0.98      0.99     97762

ROC-AUC: 0.9612436345748652
